In [1]:
import numpy as np

In [2]:
import numpy as np

# Your provided pitch feature coordinates
bottom_left_12_box_point = np.array([319, 672, 69, 39])  # Format: x, y, width, height

# Your provided homography matrix
homography_matrix = np.array([
    [0.34523, 0.24001, 511.79],
    [0.26268, 0.75285, -484.6],
    [0.0029172, 0.028532, 1]
])

# Function to transform a point using homography
def transform_point_using_homography(homography_matrix, image_point_xywh):
    """
    Transform a point from image coordinates to world coordinates using a homography matrix.
    
    Args:
        homography_matrix: 3x3 homography matrix
        image_point_xywh: Point in [x, y, width, height] format
        
    Returns:
        Transformed point in world coordinates (X, Y)
    """
    # Extract the center of the bounding box
    x, y, w, h = image_point_xywh
    
    # Option 1: Center of bounding box
    center_x = x + w / 2
    center_y = y + h / 2
    
    # Option 2: Bottom center (feet position)
    bottom_center_x = x + w / 2
    bottom_center_y = y + h
    
    # Try both center and bottom center for comparison
    points_to_try = [
        ("Center", center_x, center_y),
        ("Bottom Center", bottom_center_x, bottom_center_y)
    ]
    
    results = []
    for point_name, point_x, point_y in points_to_try:
        # Create homogeneous coordinates [x, y, 1]
        homogeneous_point = np.array([point_x, point_y, 1])
        
        # Apply the homography transformation
        transformed_point = np.dot(homography_matrix, homogeneous_point)
        
        # Convert back to 2D coordinates by dividing by the third component
        world_x = transformed_point[0] / transformed_point[2]
        world_y = transformed_point[1] / transformed_point[2]
        
        results.append((point_name, world_x, world_y))
    
    return results

# Perform the transformation and verify results
transformation_results = transform_point_using_homography(homography_matrix, bottom_left_12_box_point)

print("Verification of Homography Transformation")
print("-----------------------------------------")
print(f"Image Point (x,y,w,h): {bottom_left_12_box_point}")
print("Homography Matrix:")
for row in homography_matrix:
    print(f"  {row}")
print("\nTransformation Results:")

for point_name, world_x, world_y in transformation_results:
    print(f"{point_name} Point:")
    print(f"  World Coordinates: X = {world_x:.2f}, Y = {world_y:.2f}")

# For comparison with expected values (if you know them)
# Assuming this point might be the left goal post which should be around (41.34, 0)
# in your world coordinate system
expected_world_point = (35.85, 5.5)  # Based on your world_points list
print("\nExpected Coordinates (if this is the Left Goal Post):")
print(f"  X = {expected_world_point[0]}, Y = {expected_world_point[1]}")

# Calculate error if this is indeed the left goal post
for point_name, world_x, world_y in transformation_results:
    error_x = abs(world_x - expected_world_point[0])
    error_y = abs(world_y - expected_world_point[1])
    print(f"\nError using {point_name} Point:")
    print(f"  X error: {error_x:.2f} meters")
    print(f"  Y error: {error_y:.2f} meters")
    print(f"  Total error: {np.sqrt(error_x**2 + error_y**2):.2f} meters")

Verification of Homography Transformation
-----------------------------------------
Image Point (x,y,w,h): [319 672  69  39]
Homography Matrix:
  [3.4523e-01 2.4001e-01 5.1179e+02]
  [ 2.6268e-01  7.5285e-01 -4.8460e+02]
  [0.0029172 0.028532  1.       ]

Transformation Results:
Center Point:
  World Coordinates: X = 36.75, Y = 5.92
Bottom Center Point:
  World Coordinates: X = 36.05, Y = 6.43

Expected Coordinates (if this is the Left Goal Post):
  X = 35.85, Y = 5.5

Error using Center Point:
  X error: 0.90 meters
  Y error: 0.42 meters
  Total error: 1.00 meters

Error using Bottom Center Point:
  X error: 0.20 meters
  Y error: 0.93 meters
  Total error: 0.95 meters


In [3]:
import pandas as pd
from mplsoccer import Sbopen, Pitch


In [4]:
parser = Sbopen()
competitions = parser.competition()
df_matches = parser.match(competition_id=11, season_id=90)
df_matches

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,...,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3773386,2020-10-31,2020-10-31 21:00:00,1,1,available,available,2023-07-25 03:54:59.280826,2023-07-25 04:25:41.348202,8,...,1,Regular Season,348,Estadio de Mendizorroza,214,Spain,NaN,NaN,NaN,NaN
1,3773565,2021-01-09,2021-01-09 18:30:00,0,4,available,available,2023-07-25 03:51:37.437064,2023-07-25 04:30:16.058384,18,...,1,Regular Season,4667,Estadio Nuevo Los Cármenes,214,Spain,2602.0,Ricardo De Burgos Bengoetxea,214.0,Spain
2,3773457,2021-05-16,2021-05-16 18:30:00,1,2,available,available,2022-12-02 09:26:39.496362,2023-04-27 23:03:53.506485,37,...,1,Regular Season,342,Spotify Camp Nou,214,Spain,NaN,NaN,NaN,NaN
3,3773631,2021-02-07,2021-02-07 21:00:00,2,3,available,available,2023-07-25 03:47:44.278651,2023-07-25 03:56:34.733180,22,...,1,Regular Season,352,Estadio Benito Villamarín,214,Spain,NaN,NaN,NaN,NaN
4,3773665,2021-03-06,2021-03-06 21:00:00,0,2,available,available,2022-12-02 08:46:42.897589,2023-04-28 02:57:03.412841,26,...,1,Regular Season,4650,Estadio El Sadar,214,Spain,2402.0,Guillermo Cuadra Fernández,214.0,Spain
5,3773497,2021-04-10,2021-04-10 21:00:00,2,1,available,available,2022-12-02 09:04:21.859831,2023-04-28 01:35:16.051381,30,...,1,Regular Season,5341,Estadio Alfredo Di Stéfano,214,Spain,183.0,Jesús Gil Manzano,214.0,Spain
6,3773660,2020-12-13,2020-12-13 21:00:00,1,0,available,available,2022-12-01 14:49:02.748131,2023-04-28 06:01:10.173360,13,...,1,Regular Season,342,Spotify Camp Nou,214,Spain,2602.0,Ricardo De Burgos Bengoetxea,214.0,Spain
7,3773593,2020-09-27,2020-09-27 21:00:00,4,0,available,available,2023-07-25 04:01:57.790373,2023-07-25 04:44:03.367478,3,...,1,Regular Season,342,Spotify Camp Nou,214,Spain,2402.0,Guillermo Cuadra Fernández,214.0,Spain
8,3773466,2020-10-01,2020-10-01 21:30:00,0,3,available,available,2023-07-25 03:55:25.794505,2023-07-25 04:40:53.635540,4,...,1,Regular Season,653,Abanca-Balaídos,214,Spain,2728.0,Carlos del Cerro Grande,214.0,Spain
9,3773585,2020-10-24,2020-10-24 16:00:00,1,3,available,available,2023-07-25 03:53:35.185506,2023-07-25 04:29:21.140321,7,...,1,Regular Season,342,Spotify Camp Nou,214,Spain,2434.0,Juan Martínez Munuera,214.0,Spain


In [5]:
all_shots = []
for match_id in df_matches['match_id']:
    df_events,__,__,__ = parser.event(match_id=match_id)
    df_shots = df_events[df_events['type_name'] == 'Shot']
    all_shots.append(df_shots)
la_liga_shots = pd.concat(all_shots, ignore_index=True)


In [6]:
la_liga_shots.columns.to_list()

['id',
 'index',
 'period',
 'timestamp',
 'minute',
 'second',
 'possession',
 'duration',
 'match_id',
 'type_id',
 'type_name',
 'possession_team_id',
 'possession_team_name',
 'play_pattern_id',
 'play_pattern_name',
 'team_id',
 'team_name',
 'tactics_formation',
 'player_id',
 'player_name',
 'position_id',
 'position_name',
 'pass_recipient_id',
 'pass_recipient_name',
 'pass_length',
 'pass_angle',
 'pass_height_id',
 'pass_height_name',
 'end_x',
 'end_y',
 'body_part_id',
 'body_part_name',
 'sub_type_id',
 'sub_type_name',
 'x',
 'y',
 'under_pressure',
 'outcome_id',
 'outcome_name',
 'counterpress',
 'foul_committed_advantage',
 'foul_won_advantage',
 'aerial_won',
 'pass_switch',
 'technique_id',
 'technique_name',
 'out',
 'off_camera',
 'pass_deflected',
 'pass_cross',
 'pass_assisted_shot_id',
 'pass_shot_assist',
 'shot_one_on_one',
 'shot_statsbomb_xg',
 'end_z',
 'shot_key_pass_id',
 'goalkeeper_position_id',
 'goalkeeper_position_name',
 'foul_won_defensive',
 'fou

In [7]:
la_liga_shots = la_liga_shots[['x', 'y', 'shot_statsbomb_xg', 'body_part_name', 'outcome_name']].copy()


In [8]:
la_liga_shots

,x,y,shot_statsbomb_xg,body_part_name,outcome_name
0,108.6,28.0,0.200969,Right Foot,Off T
1,103.6,51.0,0.096384,Right Foot,Saved
2,104.3,33.9,0.098879,Left Foot,Off T
3,97.9,44.3,0.078938,Left Foot,Blocked
4,118.3,42.1,0.976192,Left Foot,Goal
...,...,...,...,...,...
834,107.5,41.8,0.035400,Head,Off T
835,111.5,49.2,0.098630,Right Foot,Saved
836,115.2,32.9,0.365302,Left Foot,Goal
837,99.2,30.4,0.032844,Right Foot,Off T


In [9]:
world_points = [
    {"Left Corner Flag": [0,0]},   # Left corner flag
    {"Top Left 18-Box": [24.85,0]},  # Top Left 18-Box
    {"Top Left 12-Box" : [35.85,0]},  # Top Left 12-Box
    {"Bottom Left 18-Box" : [24.85, 16.5]}, # Bottom Left 18-Box
    {"Bottom Left 12-Box" : [35.85, 5.5]}, # Bottom Right 12-Box
    {"Left Goal Post" : [41.34, 0]},  # Left Goal Post
    {"Right Goal Post" : [48.66, 0]},  # Right Goal Post
    {"Top Right 12-Box" : [54.15, 0]},   # Top Right 12-Box
    {"Top Right 18-Box" : [65.15, 0]},  # Top Right 18-Box
    {"Bottom Right 12-Box" : [54.15, 5.5]},  # Bottom Right 12-Box
    {"Bottom Right 18-Box" : [65.15, 16.5]},  # Bottom Right 18-Box
    {"Right Corner Flag" : [90, 0]}  # Right corner flag
    ]
shot_bomb_pitch_points = [
    {"Left Corner Flag": [0,120]},   # Left corner flag
    {"Top Left 18-Box": [18,120]},  # Top Left 18-Box
    {"Top Left 12-Box" : [30,120]},  # Top Left 12-Box
    {"Bottom Left 18-Box" : [18, 120]}, # Bottom Left 18-Box
    {"Bottom Left 12-Box" : [30, 114]}, # Bottom Right 12-Box
    {"Left Goal Post" : [36, 120]},  # Left Goal Post
    {"Right Goal Post" : [43.32, 120]},  # Right Goal Post
    {"Top Right 12-Box" : [50, 120]},   # Top Right 12-Box
    {"Top Right 18-Box" : [62, 120]},  # Top Right 18-Box
    {"Bottom Right 12-Box" : [50, 114]},  # Bottom Right 12-Box
    {"Bottom Right 18-Box" : [62, 102]},  # Bottom Right 18-Box
    {"Right Corner Flag" : [80, 120]}  # Right corner flag
    ]

In [10]:
import numpy as np
import cv2

# Flatten point dictionaries into two lists of corresponding points
world_pts = np.array([list(p.values())[0] for p in world_points], dtype=np.float32)
shotbomb_pts = np.array([list(p.values())[0] for p in shot_bomb_pitch_points], dtype=np.float32)

# Compute homography matrix from world -> shotbomb
H, status = cv2.findHomography(world_pts, shotbomb_pts)

# Function to apply homography (already defined)
def transform_point_homography(pt, H):
    pt_h = np.array([pt[0], pt[1], 1.0])
    warped = H @ pt_h
    warped /= warped[2]
    return warped[:2]

# Example: Convert a world coordinate to shotbomb system
example_world_coord = [66.51, 15.01]  # Put any coordinate you want here
converted_coord = transform_point_homography(example_world_coord, H)
print("Converted:", converted_coord)


Converted: [ 63.62159833 109.26739206]


In [11]:
la_liga_shots

,x,y,shot_statsbomb_xg,body_part_name,outcome_name
0,108.6,28.0,0.200969,Right Foot,Off T
1,103.6,51.0,0.096384,Right Foot,Saved
2,104.3,33.9,0.098879,Left Foot,Off T
3,97.9,44.3,0.078938,Left Foot,Blocked
4,118.3,42.1,0.976192,Left Foot,Goal
...,...,...,...,...,...
834,107.5,41.8,0.035400,Head,Off T
835,111.5,49.2,0.098630,Right Foot,Saved
836,115.2,32.9,0.365302,Left Foot,Goal
837,99.2,30.4,0.032844,Right Foot,Off T


In [12]:
def add_features(df):
    goal_x_center = (36.0+43.32) / 2
    goal_y = 120
    df['distance'] = np.sqrt((goal_x_center - df['x'])**2 + (goal_y-df['y'])**2)
    dx1 = df['x'] - 36.0       
    dx2 = df['x'] - 43.32      
    dy = goal_y - df['y']      

    numerator = (dx1 * dx2) + (dy**2)
    denominator = np.sqrt((dx1**2 + dy**2) * (dx2**2 + dy**2))
    df['angle'] = np.arccos(np.clip(numerator / (denominator + 1e-6), -1.0, 1.0))
    return df
la_liga_shots = add_features(la_liga_shots)
la_liga_shots.to_csv('la liga shots.csv', index=False)

In [13]:
categorical_vars = ['body_part_name', 'outcome_name']
la_liga_shots = pd.get_dummies(la_liga_shots, columns=categorical_vars, drop_first=True)
target = 'shot_statsbomb_xg'
feature_column  = [col for col in la_liga_shots.columns if col != target]
X = la_liga_shots[feature_column]
y = la_liga_shots[target]


In [14]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror', random_state = 42)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [16]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print("Test MSE:", mse)
print("Test RMSE:", rmse)

Test MSE: 0.008373819049199658
Test RMSE: 0.09150857363766336


In [17]:
training_feature_names = X.columns.tolist()
print(training_feature_names)

['x', 'y', 'distance', 'angle', 'body_part_name_Left Foot', 'body_part_name_Other', 'body_part_name_Right Foot', 'outcome_name_Goal', 'outcome_name_Off T', 'outcome_name_Post', 'outcome_name_Saved', 'outcome_name_Saved Off Target', 'outcome_name_Saved to Post', 'outcome_name_Wayward']


In [23]:
new_shot = pd.DataFrame({
    'x': [63],
    'y': [109],
    'body_part_name': 'Left Foot',
    'outcome_name': 'Goal'
})
new_shot = add_features(new_shot)
new_shot_encoded = pd.get_dummies(new_shot, columns=['body_part_name', 'outcome_name'], drop_first=True)
feature_columns = ['x', 'y'] + [col for col in new_shot_encoded.columns if col not in ['x', 'y']]
training_feature_names = ['x', 'y', 'distance', 'angle', 
                          'body_part_name_Left Foot', 'body_part_name_Other', 'body_part_name_Right Foot',
                          'outcome_name_Goal', 'outcome_name_Off T', 'outcome_name_Post', 
                          'outcome_name_Saved', 'outcome_name_Saved Off Target', 
                          'outcome_name_Saved to Post', 'outcome_name_Wayward']

# Reindex and fill missing features with zeros
new_shot_encoded = new_shot_encoded.reindex(columns=training_feature_names, fill_value=0)

print("New input vector prepared for prediction:")
print(new_shot_encoded)

New input vector prepared for prediction:
    x    y  distance     angle  body_part_name_Left Foot  \
0  63  109  25.80224  0.122808                         0   

   body_part_name_Other  body_part_name_Right Foot  outcome_name_Goal  \
0                     0                          0                  0   

   outcome_name_Off T  outcome_name_Post  outcome_name_Saved  \
0                   0                  0                   0   

   outcome_name_Saved Off Target  outcome_name_Saved to Post  \
0                              0                           0   

   outcome_name_Wayward  
0                     0  


In [26]:
new_shot_encoded['body_part_name_Left Foot'] = 1
new_shot_encoded['outcome_name_Goal'] = 1

In [27]:
xg_prediction = model.predict(new_shot_encoded)

print("Predicted xG:", xg_prediction[0])


Predicted xG: 0.056663685
